In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from matplotlib import colors
import pickle
import math
from collections import Counter

In [2]:
#data = pd.read_pickle('..\output\data.pkl')
def download_data(N,name_file):
    path = '..\output\X' + str(N) + '\\'
    path_name_file = str(path) + str(name_file) + '.pkl'
    file = pd.read_pickle(path_name_file)
    return file 

def upload_data(N,file,name_file):
    path = '..\output\X' + str(N) + '\\'
    path_name_file_plk = str(path) + str(name_file) + '.pkl'
    path_name_file_csv = str(path) + str(name_file) + '.csv'    
    file.to_pickle(path_name_file_plk)
    file.to_csv(path_name_file_csv, sep='\t', encoding='utf-8')
    return path_name_file_plk 

In [3]:
def compute_baseline(N_conv_min_speaker,n_min_words, size_data, n_directory):
    
    conv_tab = pd.read_csv("..\conv_tab.csv")
    caller_tab = pd.read_csv("..\caller_tab.csv")
    call_con_tab = pd.read_csv("..\call_con_tab.csv")
    topic_tab = pd.read_csv("..\Topic_tab.csv")    
    name_file = 'speech_rate_minword' + str(n_min_words) + 'std_' + 'size%_'+ str(size_data*100)+ '_' + str(N_conv_min_speaker) + 'conv_speaker'
    speech_rate_filter = download_data(n_directory, name_file)
    
    caller_tab_selected = caller_tab[['id_speaker','sex','age','geography','level_study']]
    caller_tab_selected.rename(columns={'id_speaker': 'id_interlocutor', 'sex': 'sex_int', 'age':'age_int', 'geography':'geography_int','level_study':'level_study_int' }, inplace=True)
    speaker_speech_rate_std_filt_conv = speech_rate_filter.merge(caller_tab_selected, how='left', on = 'id_interlocutor')
    speaker_speech_rate_std_filt_conv['age_int'] = 1991 - speaker_speech_rate_std_filt_conv['age_int']
    df = speaker_speech_rate_std_filt_conv[['id_speaker', 'id_conv']]
#speech_rate_prova = speaker_speech_rate_std_filt_conv 
  #  df = df.drop_duplicates()
    
    name_file_tot = 'speech_rate_minword' + str(n_min_words) + 'std'
    speech_rate = download_data(n_directory, name_file_tot)
    
    progress = (1/len(df['id_speaker']))
    for speaker in df['id_speaker']: 
        labels = df[df['id_speaker'] == speaker].index.values.tolist()
        #print(speaker)
        progress = progress + (1/len(df['id_speaker']))
        
        for indice in labels:
            
            
            df_indice = speech_rate[speech_rate['id_speaker'] == speaker]
            conv_to_drop = []
            conv_to_drop.append(df['id_conv'].ix[indice])
            
            df_indice = df_indice[(~df_indice['id_conv'].isin(conv_to_drop))]
        
            baseline = df_indice.m_log_pointw_speech_rate.mean()
            baseline_nolog = df_indice.mean_pointwise_speech_rate.mean()
            
            speaker_speech_rate_std_filt_conv.at[indice, 'baseline'] = baseline
            speaker_speech_rate_std_filt_conv.at[indice, 'baseline_nolog'] = baseline_nolog
        update_progress(progress)   
    speaker_bas_int = speaker_speech_rate_std_filt_conv[['id_conv','id_caller', 'baseline', 'baseline_nolog']]
    speaker_bas_int['id_caller'] = speaker_bas_int['id_caller'].replace(['A'], 1)
    speaker_bas_int['id_caller'] = speaker_bas_int['id_caller'].replace(['B'], 0)
    speaker_bas_int = speaker_bas_int.sort_values(by = ['id_conv','id_caller'], ascending = [True,True])
    speaker_bas_int.index = speaker_speech_rate_std_filt_conv.index
    speaker_speech_rate_std_filt_conv = pd.merge(speaker_speech_rate_std_filt_conv, speaker_bas_int, left_index=True, right_index=True)
    del speaker_speech_rate_std_filt_conv['id_conv_y']
    del speaker_speech_rate_std_filt_conv['id_caller_y']
    speaker_speech_rate_std_filt_conv.rename(columns={ 'id_caller_x': 'id_caller',  'id_conv_x': 'id_conv', 'baseline_x': 'baseline_S', 'baseline_y': 'baseline_I', 'baseline_nolog_x': 'baseline_no_log_S', 'baseline_nolog_y': 'baseline_no_log_I'}, inplace=True)
    
    name_file = 'speech_rate_minword' + str(n_min_words) + 'std_' + 'size%_'+ str(size_data*100)+ '_' + str(N_conv_min_speaker) + 'conv_speaker_baseline'
    #name_file = 'speech_rate_minword' + str(n_min_words) + 'std_' + str(N_conv_min_speaker) + 'conv_speaker_baseline'
    #name_file = 'speech_rate_std_filt_conv' + str(N_conv_min_speaker)
    upload_data(n_directory,speaker_speech_rate_std_filt_conv,name_file)  
    return speaker_speech_rate_std_filt_conv

In [4]:
 import time, sys

# update_progress() : Displays or updates a console progress bar
## Accepts a float between 0 and 1. Any int will be converted to a float.
## A value under 0 represents a 'halt'.
## A value at 1 or bigger represents 100%
def update_progress(progress):
    barLength = 20 # Modify this to change the length of the progress bar
    status = ""
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
        status = "error: progress var must be float\r\n"
    if progress < 0:
        progress = 0
        status = "Halt...\r\n"
    if progress >= 1:
        progress = 1
        status = "Done...\r\n"
    block = int(round(barLength*progress))
    text = "\rPercent: [{0}] {1}% {2}".format( "="*block + " "*(barLength-block), progress*100, status)
    sys.stdout.write(text)
    sys.stdout.flush()

In [ ]:
def std_baseline(N_conv_min_speaker,n_min_words, n_directory):
    from sklearn import preprocessing
   
    name_file = 'speech_rate_minword' + str(n_min_words) + 'std_' + 'size%_'+ str(size_data*100)+ '_' + str(N_conv_min_speaker) + 'conv_speaker_baseline'
    speaker_speech_rate_std_filt_conv = download_data(n_directory, name_file)
    
    #standardize age
    x = speaker_speech_rate_std_filt_conv[['age_int','age']].values #returns a numpy array
    #min_max_scaler_age = preprocessing.MinMaxScaler(feature_range=(0,6))
    #x_scaled = min_max_scaler_age.fit_transform(x)
    x_scaled = preprocessing.scale(x)
    numeric_speech_rate_std = pd.DataFrame(x_scaled, index=speaker_speech_rate_std_filt_conv.index)
    numeric_speech_rate_std.columns = ['age_int','age']
    speaker_speech_rate_std_filt_conv['age_int_std']  = numeric_speech_rate_std['age_int']
    speaker_speech_rate_std_filt_conv['age_std']  = numeric_speech_rate_std['age']

    x = speaker_speech_rate_std_filt_conv[['id_topic','id_speaker','id_conv']].values #returns a numpy array
    #min_max_scaler = preprocessing.MinMaxScaler(feature_range=(-10,10))
    #x_scaled = min_max_scaler.fit_transform(x)
    x_scaled = preprocessing.scale(x)
    numeric_speech_rate_std = pd.DataFrame(x_scaled, index=speaker_speech_rate_std_filt_conv.index)
    numeric_speech_rate_std.columns = ['id_topic','id_interlocutor','id_conv']
    speaker_speech_rate_std_filt_conv['id_topic_std']  = numeric_speech_rate_std['id_topic']
    speaker_speech_rate_std_filt_conv['id_interlocutor_std']  = numeric_speech_rate_std['id_interlocutor']
    speaker_speech_rate_std_filt_conv['id_conv_std']  = numeric_speech_rate_std['id_conv']
    
    x = speaker_speech_rate_std_filt_conv[['baseline_S','baseline_I']].values #returns a numpy array
    #min_max_scaler_age = preprocessing.MinMaxScaler(feature_range=(0,6))
    #x_scaled = min_max_scaler_age.fit_transform(x)
    x_scaled = preprocessing.scale(x)
    numeric_speech_rate_std = pd.DataFrame(x_scaled, index=speaker_speech_rate_std_filt_conv.index)
    numeric_speech_rate_std.columns = ['baseline_S','baseline_I']
    speaker_speech_rate_std_filt_conv['baseline_S_std']  = numeric_speech_rate_std['baseline_S']
    speaker_speech_rate_std_filt_conv['baseline_I_std']  = numeric_speech_rate_std['baseline_I']   
    
    
    speaker_speech_rate_std_filt_conv['sex'] = speaker_speech_rate_std_filt_conv['sex'].replace(['FEMALE'], 0)
    speaker_speech_rate_std_filt_conv['sex'] = speaker_speech_rate_std_filt_conv['sex'].replace(['MALE'], 1)
    speaker_speech_rate_std_filt_conv['sex_int'] = speaker_speech_rate_std_filt_conv['sex_int'].replace(['FEMALE'], 0)
    speaker_speech_rate_std_filt_conv['sex_int'] = speaker_speech_rate_std_filt_conv['sex_int'].replace(['MALE'], 1)

    #x = speaker_speech_rate_std_filt_conv[['sex','sex_int']].values #returns a numpy array
    #x_scaled = preprocessing.scale(x)
    #numeric_speech_rate_std = pd.DataFrame(x_scaled, index=speaker_speech_rate_std_filt_conv.index)
    #numeric_speech_rate_std.columns = ['sex','sex_int']
    #speaker_speech_rate_std_filt_conv['sex']  = numeric_speech_rate_std['sex']
    #speaker_speech_rate_std_filt_conv['sex_int']  = numeric_speech_rate_std['sex_int']    
    
    
    upload_data(n_directory,speaker_speech_rate_std_filt_conv,name_file)  
    return speaker_speech_rate_std_filt_conv